In [11]:
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import requests
import re

In [12]:
class Batchurls():

  def __init__(self,*pages):
    templete = 'https://answers.justia.com/questions/answered/california?page='
    self.docs = [templete + str(i) for i in range(pages[0],pages[1]+1)]

    f = lambda x: BeautifulSoup(self.get_html(x))
    self.docs = list(map(f,self.docs))

  def get_html(self, url):
    x = requests.get(url)
    return x.text

  def update_doc(self):
    f = lambda x: BeautifulSoup(self.get_html(x))
    self.docs = list(map(f,self.postlinks))

  def get_qlink(self):
    self.postlinks = []
    templete = 'https://answers.justia.com'

    for doc in self.docs:
      for s in doc.find_all('strong'):
          if re.match('(Q:)',str(s.next_element)):
            link = templete + str(s.find('a')['href'])
            self.postlinks.append(link)

In [91]:
#This part will take some time depedning on the amount of pages you want to scrape
urls = Batchurls(2,12) #Enter page number from 2 to some number more than 2(I didn't check the end of this website, but I was able to find over 500 page)
urls.get_qlink()
urls.update_doc()

In [92]:
dic = {'question':[],'answer':[]}
for doc in urls.docs:
  string_question = doc.find('title').text
  string = doc.find_all('p')[0].text
  if re.search('(A\:)',string):
    string_answer = string
  else:
    string_question += string
    string_answer = doc.find_all('p')[1].text

  dic['question'].append(string_question)
  dic['answer'].append(string_answer)


In [93]:
df = pd.DataFrame(dic)

In [97]:
df.to_json('data.json',orient='records')